In [5]:
import cv2
import numpy as np
import os


def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    corner=toIntTuple(corner)
    corner=toIntTuple(corner)
    a=toIntTuple(tuple(imgpts[0].ravel()))
    b=toIntTuple(tuple(imgpts[1].ravel()))
    c=toIntTuple(tuple(imgpts[2].ravel()))
    img = cv2.line(img, corner, a , (255,0,0), 5)
    img = cv2.line(img, corner, b , (0,255,0), 5)
    img = cv2.line(img, corner, c , (0,0,255), 5)
    return img

def toIntTuple(tup):
    x=round(tup[0])
    y=round(tup[1])
    return(x,y)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)



# Load the saved file

noSuccessfulInput = True
while noSuccessfulInput:
    print("TEST")
    try:
        fileInt = int(input("Enter Calibration file number:\t"))
    except:
        print("Integer only")
        
    try:
        data = np.load("calibration"+str(fileInt)+".npz")
    except:
        print("File not found")
    else:
        print("File found")
        noSuccessfulInput=False
        
        
# Access the individual arrays
mtx = data['mtx']
dist = data['dist']
rvecs = data['rvecs']
tvecs = data['tvecs']

CAM_ENUM = int(data['CAM_ENUM'])
CAM_WIDTH = int(data['CAM_WIDTH'])
CAM_HEIGHT = int(data['CAM_HEIGHT'])

camera= cv2.VideoCapture(CAM_ENUM,cv2.CAP_DSHOW)
camera.set(cv2.CAP_PROP_FRAME_WIDTH, CAM_WIDTH)  # Set the width
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, CAM_HEIGHT)  # Set the height


while True:
    absImg=np.zeros((480, 640, 3))
#q    absImg=np.zeros((CAM_WIDTH,CAM_HEIGHT,3))
    noSuccessfulInput, img = camera.read()
    if noSuccessfulInput:        
        img = cv2.flip(img,1)
        cv2.imshow('img',img)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (7,6),None)
        
        if ret == True:
            try:
                corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
                # Find the rotation and translation vectors.
                ret,rvecs, tvecs = cv2.solvePnP(objp, corners2, mtx, dist)
                # project 3D points to image plane
                imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
                img = draw(img,corners2,imgpts)
                absImg = draw(absImg,corners2,imgpts)
                cv2.imshow('Pose Abstraction',absImg)
            except:
                print("Exception")
        cv2.imshow('img',img)


    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        camera.release()
        break
cv2.destroyAllWindows()

TEST
File found
